## Implementação de novas features usando o dataframe com targets atualizados

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle("/data/challenges/hackathon_bigdata_2025/processed_data/merged_transactions_with_targets.pkl")

In [3]:
data

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,marca,fabricante,descricao,gross_value,net_value,gross_profit,discount,taxes,quantidade,target
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,Michelob Ultra,AB Anheuser Busch Inc,MICHELOB ULTRA 2/12/12 CN,54.299999,49.740486,16.224486,0.000000,4.559516,2.0,Not available
1,1001371918471115422,30175.0,1120490062981954254,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 2/12/8 CN,262.350006,234.992905,62.210903,53.099998,27.357101,18.0,Not available
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 4/6/16 CN,37.950001,31.870647,8.182648,5.100000,6.079354,2.0,Not available
3,1001371918471115422,30175.0,4353552881410365573,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 2/15/12 CN,138.250000,118.302116,33.567116,2.800000,19.947886,7.0,Not available
4,1001371918471115422,30175.0,4797439216678436447,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light Lime,AB Anheuser Busch Inc,BUD LIGHT LIME 2/12/12 CN,26.750000,24.470242,7.524241,0.000000,2.279758,1.0,Not available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6166028,996297376394222790,30035.0,9024979450937837276,5,12,2022,Convenience,Non-Alcohol,Water,Off Premise,...,Essentia Water,Essentia Water LLC,ESSENTIA WATER 24/20 NR,24.600000,24.600000,8.560000,0.000000,0.000000,1.0,Not available
6166029,997907800111849739,80205.0,3516587220187345473,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Natural Ice,AB Anheuser Busch Inc,NATURAL ICE 15/25 CN,220.725006,218.615631,51.665630,24.750000,2.109375,9.0,Not available
6166030,997907800111849739,80205.0,4451855961536069898,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Budweiser,AB Anheuser Busch Inc,BUDWEISER 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available
6166031,997907800111849739,80205.0,7029448080469668619,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available


In [4]:
# Custo da mercadoria = gross_value / quantidade (para o cliente)
# Custo para da mercadoria para o pdv = net_value - gross_profit

# Valor total pago pelo cliente = gross_value; O preço original seria (gross_value + discount)
# Net_value: faturamento da venda descontando impostos
# Gross_value = net_value - custo_de_mercadoria

In [5]:
ids = data['quantidade'] > 0 # update only where quantidade > 0 to avoid division by zero
data['buyer_price_per_unit'] = 0
data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
data['pdv_cost_per_unit'] = 0
data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()

/tmp/ipykernel_653421/3990647784.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[27.14999962 14.57500034 18.97500038 ... 32.85       32.85
 37.20000076]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
/tmp/ipykernel_653421/3990647784.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[16.75800014  9.59900008 11.84399986 ... 23.35899963 23.35899963
 26.40600014]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()


In [6]:
data['pdv_produto'] = data['pdv'].astype(str) + '_' + data['produto'].astype(str)

In [7]:
test_values = np.arange(1, 60, 1)

#for i in test_values:
#    print(f"i: {i}, count: {(data['pdv_produto'].value_counts() >= i).sum()}")


In [8]:
totais = (
    data
    .groupby(["pdv_produto"], as_index=False)["quantidade"]
    .sum()
    .rename(columns={"quantidade": "total_vendido"})
)

limite_linhas = 1500000
n_semanas = 5
max_pares = limite_linhas // n_semanas   # 300k

top_pares = (
    totais
    .sort_values("total_vendido", ascending=False)
    .head(max_pares)[["pdv_produto"]]
)

In [9]:
top_pares

,pdv_produto
46457,1365839146278641704_6608102981224815905
46449,1365839146278641704_1564721138677191237
655855,6434414063399492998_3652281557572916499
655869,6434414063399492998_7660365129551200715
571155,5658815032567467999_7660365129551200715
...,...
326613,3699409303667791145_1637530677008862181
10745,1082332438014605784_1532487856615257223
476864,4919303698115372961_6450782548967375358
476865,4919303698115372961_6470991684491145826


In [10]:
filtered = data.merge(top_pares, on=["pdv_produto"], how="inner")
filtered

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,gross_value,net_value,gross_profit,discount,taxes,quantidade,target,buyer_price_per_unit,pdv_cost_per_unit,pdv_produto
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,54.299999,49.740486,16.224486,0.000000,4.559516,2.0,Not available,27.150000,16.758000,1001371918471115422_1029370090212151375
1,1001371918471115422,30175.0,1120490062981954254,1,1,2022,Convenience,Package,Lager,Off Premise,...,262.350006,234.992905,62.210903,53.099998,27.357101,18.0,Not available,14.575000,9.599000,1001371918471115422_1120490062981954254
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,37.950001,31.870647,8.182648,5.100000,6.079354,2.0,Not available,18.975000,11.844000,1001371918471115422_2239307647969388381
3,1001371918471115422,30175.0,4353552881410365573,1,1,2022,Convenience,Package,Lager,Off Premise,...,138.250000,118.302116,33.567116,2.800000,19.947886,7.0,Not available,19.750000,12.105000,1001371918471115422_4353552881410365573
4,1001371918471115422,30175.0,5913249594472983850,1,1,2022,Convenience,Package,Lager,Off Premise,...,102.025002,91.386131,24.193130,20.650000,10.638873,7.0,Not available,14.575000,9.599000,1001371918471115422_5913249594472983850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336082,990571711439878263,80031.0,4990795394027517284,5,12,2022,Package/Liquor,Distilled Spirits,Vodka,Off Premise,...,71.040001,65.616890,8.536886,0.000000,5.423112,12.0,Not available,5.920000,4.756667,990571711439878263_4990795394027517284
3336083,996297376394222790,30035.0,3407816598812292920,5,12,2022,Convenience,Package,Lager,Off Premise,...,58.799999,52.863129,14.819129,0.000000,5.936870,2.0,Not available,29.400000,19.022000,996297376394222790_3407816598812292920
3336084,997907800111849739,80205.0,3516587220187345473,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,220.725006,218.615631,51.665630,24.750000,2.109375,9.0,Not available,24.525001,18.550000,997907800111849739_3516587220187345473
3336085,997907800111849739,80205.0,4451855961536069898,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available,32.850000,23.359000,997907800111849739_4451855961536069898


In [11]:
#values = (data.pdv_produto.value_counts() >= 40).values
#filtered_data = data[data['pdv_produto'].isin(data.pdv_produto.value_counts()[values].index)]

In [12]:
# if filtered_data.mes != 12, then filter all data with not available target
without_dec = filtered[filtered['mes'] != 12]; with_dec = filtered[filtered['mes'] == 12]
without_dec = without_dec[without_dec.target != 'Not available']

filtered = pd.concat([without_dec, with_dec])

In [13]:
filtered.pdv_produto.nunique()

186042

In [14]:
filtered_train = filtered[filtered.mes.isin([8, 9])]
filtered_val = filtered[filtered.mes.isin([10, 11])]

In [15]:
min_month_train = filtered_train.mes.min()
filtered_train['mes'] = filtered_train.mes - min_month_train
filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana

min_month_val = filtered_val.mes.min()
filtered_val.mes = filtered_val.mes - min_month_val
filtered_val['semana_new'] = filtered_val.mes * 5 + filtered_val.semana

/tmp/ipykernel_653421/316573106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['mes'] = filtered_train.mes - min_month_train
/tmp/ipykernel_653421/316573106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana
/tmp/ipykernel_653421/316573106.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [16]:
filtered_train.semana_new.unique(), filtered_val.semana_new.unique()

(array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32),
 array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32))

In [17]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_653421/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [18]:
#filtered_train[filtered_train.pdv_produto == '1001371918471115422_3407816598812292920'] # Para esse caso, há apenas um target disponível no mês seguinte

In [19]:
filtered_train.pdv_produto.value_counts()

pdv_produto
7200116106667687476_8625590539951587748    10
1057223373529991552_8030392857471983373    10
1057223373529991552_7890016101383296727    10
1057223373529991552_700484830392455456     10
2866379461020353700_519576013096278235     10
                                           ..
7573912542927693570_2262431355921878551     1
7573912542927693570_2000299989413890957     1
7573912542927693570_1471722963249789482     1
7516711027968470290_2525510189893188235     1
746282210001185599_8960695008373210791      1
Name: count, Length: 101946, dtype: int64

In [20]:
filtered_train[filtered_train.pdv_produto == '4440977747777161208_1860061817666925715'].sort_values(by=['mes', 'semana']) # Para esse caso, há targets disponíveis para as 5 semanas do mês seguinte

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,gross_profit,discount,taxes,quantidade,target,buyer_price_per_unit,pdv_cost_per_unit,pdv_produto,semana_new,semana_mes
425498,4440977747777161208,80550.0,1860061817666925715,1,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,21.0,19.504000,15.359000,4440977747777161208_1860061817666925715,1,0_1
1151414,4440977747777161208,80550.0,1860061817666925715,2,0,2022,Grocery,Package,Lager,Off Premise,...,83.265051,87.149998,3.78,21.0,7.0,19.504000,15.358998,4440977747777161208_1860061817666925715,2,0_2
2037417,4440977747777161208,80550.0,1860061817666925715,3,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,14.0,19.504000,15.359000,4440977747777161208_1860061817666925715,3,0_3
2802975,4440977747777161208,80550.0,1860061817666925715,4,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,6.0,19.504000,15.359000,4440977747777161208_1860061817666925715,4,0_4
3214240,4440977747777161208,80550.0,1860061817666925715,5,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,7.0,19.504000,15.359000,4440977747777161208_1860061817666925715,5,0_5
487338,4440977747777161208,80550.0,1860061817666925715,1,1,2022,Grocery,Package,Lager,Off Premise,...,94.730988,63.000000,3.78,21.0,28.0,20.049999,15.359000,4440977747777161208_1860061817666925715,6,1_1
1273485,4440977747777161208,80550.0,1860061817666925715,2,1,2022,Grocery,Package,Lager,Off Premise,...,31.577011,21.000000,1.26,7.0,7.0,20.050001,15.359000,4440977747777161208_1860061817666925715,7,1_2
2098237,4440977747777161208,80550.0,1860061817666925715,3,1,2022,Grocery,Package,Lager,Off Premise,...,72.604004,46.200001,2.52,14.0,7.0,20.725000,15.359000,4440977747777161208_1860061817666925715,8,1_3
2864400,4440977747777161208,80550.0,1860061817666925715,4,1,2022,Grocery,Package,Lager,Off Premise,...,31.116002,19.800000,1.08,6.0,14.0,20.724999,15.359000,4440977747777161208_1860061817666925715,9,1_4
3253917,4440977747777161208,80550.0,1860061817666925715,5,1,2022,Grocery,Package,Lager,Off Premise,...,36.302002,23.100000,1.26,7.0,14.0,20.725000,15.359000,4440977747777161208_1860061817666925715,10,1_5


In [21]:
filtered_train.pdv_produto.value_counts()

pdv_produto
7200116106667687476_8625590539951587748    10
1057223373529991552_8030392857471983373    10
1057223373529991552_7890016101383296727    10
1057223373529991552_700484830392455456     10
2866379461020353700_519576013096278235     10
                                           ..
7573912542927693570_2262431355921878551     1
7573912542927693570_2000299989413890957     1
7573912542927693570_1471722963249789482     1
7516711027968470290_2525510189893188235     1
746282210001185599_8960695008373210791      1
Name: count, Length: 101946, dtype: int64

In [22]:
(filtered_train.pdv_produto.value_counts() == 10).sum()

np.int64(179)

In [23]:
(filtered_train.target == 'Not available').sum()

np.int64(0)

In [24]:
label_week1 = '1_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '1_2'
label_week3 = '1_3'
label_week4 = '1_4'
label_week5 = '1_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  30176
Número de labels disponíveis para a semana 2:  41329
Número de labels disponíveis para a semana 3:  29503
Número de labels disponíveis para a semana 4:  30877
Número de labels disponíveis para a semana 5:  1615


In [25]:
def aggr_df(df, categorical_features=[
    'label', 'categoria_pdv', 'categoria'], temporal_features=['quantidade', 'discount', 'gross_value', 'net_value', 'buyer_price_per_unit', 'pdv_cost_per_unit'], week=1, month=1):
    filtered_df = df.copy()

    # Add all features and mantain target column
    first = False
    for feature in temporal_features:
        if feature not in filtered_df.columns:
            raise ValueError(f"Feature {feature} not found in dataframe columns")
        # Aggregate temporal features by pdv_produto and semana_mes using sum
        # Create pivot tables for each temporal feature
        # For example, for quantidade, create features such as "quantidade_08_1", "quantidade_08_2", ..., "quantidade_09_4"
        filtered_X_agg = filtered_df.pivot_table(index='pdv_produto', columns='semana_mes', values=feature, aggfunc='sum')
        filtered_X_agg.columns = [f'{feature}_{col}' for col in filtered_X_agg.columns]
        filtered_X_agg = filtered_X_agg.reset_index()
        if not first:
            first = True
            concat_df = filtered_X_agg.copy()
        else:
            concat_df = concat_df.merge(filtered_X_agg, on='pdv_produto', how='left')

    # For categorical features, create features such as "label", "categoria_pdv", "categoria" using the first value found for that pdv_produto
    categorical_agg = filtered_df.groupby('pdv_produto').agg({
        feature: 'first' for feature in categorical_features
    }).reset_index()
    # Merge all features into a single dataframe
    concat_df =concat_df.merge(categorical_agg, on='pdv_produto', how='left')
    # Get dummies for categorical features
    concat_df = pd.get_dummies(concat_df, columns=categorical_features, drop_first=True)

    # Create 5 target columns for each row; Each target column corresponds to the target for each week (1 to 5), if available, if not fill with None
    for w in range(1, 6):
        label_week = f'{month}_{w}'
        target_agg = filtered_df[filtered_df.semana_mes == label_week][['pdv_produto', 'target']].rename(columns={'target': f'target_week{w}'})
        
        #return target_agg
        concat_df = concat_df.merge(target_agg, on='pdv_produto', how='left')
        # If there are multiple targets for the same pdv_produto, take the first one
        concat_df = concat_df.drop_duplicates(subset=['pdv_produto'], keep='first')
        # If no target is available for that week, fill with None
        if f'target_week{w}' not in concat_df.columns:
            concat_df[f'target_week{w}'] = "Not available"

    # Fill target_df Nan Values with "Not available"
    concat_df[['target_week1', 'target_week2', 'target_week3', 'target_week4', 'target_week5']] = concat_df[
        ['target_week1', 'target_week2', 'target_week3', 'target_week4', 'target_week5']].fillna("Not available")
    concat_df = concat_df.fillna(0)

    return concat_df

In [26]:
#values = (filtered_val.pdv_produto.value_counts() >= 8).values
#filtered_val = filtered_val[filtered_val['pdv_produto'].isin(filtered_val.pdv_produto.value_counts()[values].index)]

In [27]:
features_train = aggr_df(filtered_train)
features_val = aggr_df(filtered_val)

In [28]:
filtered_train[filtered_train.pdv_produto == '1001371918471115422_3407816598812292920'] # Para esse caso, há apenas um target disponível no mês seguinte

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,gross_profit,discount,taxes,quantidade,target,buyer_price_per_unit,pdv_cost_per_unit,pdv_produto,semana_new,semana_mes
399397,1001371918471115422,30175.0,3407816598812292920,1,0,2022,Convenience,Package,Lager,Off Premise,...,14.159128,0.0,5.93687,2.0,1.0,28.299999,18.252,1001371918471115422_3407816598812292920,1,0_1


In [29]:
features_train[features_train.pdv_produto == "1001371918471115422_3407816598812292920"]

,pdv_produto,quantidade_0_1,quantidade_0_2,quantidade_0_3,quantidade_0_4,quantidade_0_5,quantidade_1_1,quantidade_1_2,quantidade_1_3,quantidade_1_4,...,categoria_Distilled Spirits,categoria_Draft,categoria_Non-Alcohol,categoria_Package,categoria_Wine,target_week1,target_week2,target_week3,target_week4,target_week5
9,1001371918471115422_3407816598812292920,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,Not available,Not available,Not available,Not available,Not available


In [30]:
features_train[features_train.pdv_produto == '4440977747777161208_1860061817666925715']

,pdv_produto,quantidade_0_1,quantidade_0_2,quantidade_0_3,quantidade_0_4,quantidade_0_5,quantidade_1_1,quantidade_1_2,quantidade_1_3,quantidade_1_4,...,categoria_Distilled Spirits,categoria_Draft,categoria_Non-Alcohol,categoria_Package,categoria_Wine,target_week1,target_week2,target_week3,target_week4,target_week5
41772,4440977747777161208_1860061817666925715,7.0,21.0,7.0,7.0,7.0,21.0,7.0,14.0,6.0,...,False,False,False,True,False,28.0,7.0,7.0,14.0,14.0


In [31]:
def combine_cat_cols(train, val):
    for col in val.columns:
        if col not in train.columns:
            train[col] = 0

    for col in train.columns:
        if col not in val.columns:
            val[col] = 0

    return train, val

features_train, features_val = combine_cat_cols(features_train, features_val)

### Agora temos um dataframe de features que organiza todos os targets existentes para um dado pdv_produto.

**Descrição detalhadada:**
1. Ao selecionarmos uma janela de tempo (e.g. 2 meses), extraímos features associadas a cada semana desse período, para que o modelo possa ter uma noção da temporalidade; A feature quantidade 1_5 representa o período mais próximo, enquanto a quantidade0_1, o mais antigo;
2. Cada par pdv_produto presente nesse período pode ou não ter novas instâncias em semanas do mês seguinte; Por isso, organizamos o target dessa forma: target_week1, target_week2...;
3. Essa estrutura permite que treinemos 5 modelos, cada um dedicado a inferir em uma semana do mês futuro aos dados disponibilizados; 
4. Contamos com features de série temporal e features categóricas, que vão ajudar o modelo a aprender características associadas ao pdv e ao produto; Espera-se que o modelo, ao receber em avaliação um novo par pdv-produto, consiga através das features categóricas gerar uma boa predição. Ou seja, o modelo olha para as características da loja e produto para se contextualizar, e então usa features de time series para identificar um padrão dentro desse contexto. (Hipótese)

### Experimento de treinamento:

Em um notebook anterior, experimentamos treinar usando apenas dados de um mês. Nesse, vamos testar uma abordagem que considera mais tempo, e comparar com resultados em ao usar apenas um mês.

**EXP1:**
Treino contém dados dos meses [8, 9]; Validação contém dados do mês [10,11];

**EXP2:**
Treino contém dados dos meses [9]; Validação contém dados do mês [11];

In [34]:
import lightgbm as lgb
import numpy as np

learning_rate = [1e-1, 1e-2, 1e-3]
n_estimators = [200]
max_depth = [10, 30, 50]
num_leaves = [10, 20, 30]
min_child_samples = [5, 10, 20]
reg_alpha = [0.01, 0.1]
reg_lambda = [0.01, 0.1]

def wmape_metric(y_true, y_pred):
    """
    Calcula o Weighted Mean Absolute Percentage Error (wMAPE) para o LightGBM.
    A função precisa retornar uma lista de tuplas no formato:
    (nome_da_metrica, valor_da_metrica, is_higher_better)
    """
    sum_of_true_values = np.sum(np.abs(y_true))
    if sum_of_true_values == 0:
        return [('wmape', 0.0, False)] # Se não há vendas, o erro é zero.

    wmape_score = np.sum(np.abs(y_true - y_pred)) / sum_of_true_values
    
    return [('wmape', wmape_score, False)]

def train_model(X_train, y_train, X_val=None, y_val=None, params = None):
    lgbm = lgb.LGBMRegressor(
        objective='regression',
        metric=None,
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        random_state=41,
        max_depth=params['max_depth'],
        num_leaves=params['num_leaves'],
        min_child_samples=params['min_child_samples'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        n_jobs = 16,
        #num_iterations = 50 if X_val is None else 500,
    )

    lgbm.fit(
        X_train, 
        y_train,
        eval_metric=wmape_metric,
        eval_set=[(X_val, y_val)] if X_val is not None and y_val is not None else None,
        callbacks=[lgb.early_stopping(500, verbose=True)] if X_val is not None and y_val is not None else None,
        #categorical_feature=['label', 'categoria_pdv', 'categoria']
    )
    
    return lgbm

def optimize_hyperparameters(X_train, y_train, X_val, y_val):
    best_wmape = float('inf')
    best_params = None
    best_model = None

    for lr in learning_rate:
        for n_est in n_estimators:
            for md in max_depth:
                for nl in num_leaves:
                    for mcs in min_child_samples:
                        for ra in reg_alpha:
                            for rl in reg_lambda:
                                params = {
                                    'learning_rate': lr,
                                    'n_estimators': n_est,
                                    'max_depth': md,
                                    'num_leaves': nl,
                                    'min_child_samples': mcs,
                                    'reg_alpha': ra,
                                    'reg_lambda': rl
                                }
                                model = train_model(X_train, y_train, X_val, y_val, params)
                                y_pred = model.predict(X_val)
                                wmape = np.sum(np.abs(y_val - y_pred)) / np.sum(np.abs(y_val))

                                if wmape < best_wmape:
                                    best_wmape = wmape
                                    best_params = params
                                    best_model = model
                                    print(f"New best wMAPE: {best_wmape:.4f} with params: {best_params}")

    return best_model, best_params, best_wmape

def calculate_results_for_each_model(train, val, group_size=5):
    best_models = []
    best_params_list = []
    best_wmapes = []

    target_cols = [f'target_week{i+1}' for i in range(5)]
    #X_train = train.drop(columns=target_cols + ['pdv_produto'])
    #X_val = val.drop(columns=target_cols + ['pdv_produto'])


    for week in range(group_size):
        print(f"Optimizing hyperparameters for week {week + 1}")
        train_week = train[train[target_cols[week]] != "Not available"]
        val_week = val[val[target_cols[week]] != "Not available"]

        #return train_week
        X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
        y_train_week = train_week[target_cols[week]].astype(float)
        X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
        y_val_week = val_week[target_cols[week]].astype(float)

        
        best_model, best_params, best_wmape = optimize_hyperparameters(X_train_week, y_train_week, X_val_week, y_val_week)
        
        best_models.append(best_model)
        best_params_list.append(best_params)
        best_wmapes.append(best_wmape)

    return best_models, best_params_list, best_wmapes

In [206]:
#train_week

In [207]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12896
[LightGBM] [Info] Number of data points in the train set: 30176, number of used features: 102
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.613319
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[406]	valid_0's l2: 88.1063	valid_0's wmape: 0.407868
New best wMAPE: 0.4079 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-

KeyboardInterrupt: 

In [53]:
best_wmapes # Melhor resultado para a semana 1

[np.float64(0.4076251125053156)]

In [208]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 10,
 'num_leaves': 20,
 'min_child_samples': 5,
 'reg_alpha': 0.01,
 'reg_lambda': 0.1}

In [32]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 20,
 'min_child_samples': 5,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [35]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))

Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12896
[LightGBM] [Info] Number of data points in the train set: 30176, number of used features: 102
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.613319
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[114]	valid_0's l2: 89.8991	valid_0's wmape: 0.410865
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003496 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13413
[Li

In [36]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.4472296841275605), np.float64(0.0457867232762134))

In [37]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4434812089131694), np.float64(0.0442613376378251))

Resultados EXP1: 
WMAPE = 46.23% +- 3.70%

In [213]:
filtered_train = filtered[filtered.mes.isin([9])]
filtered_val = filtered[filtered.mes.isin([11])]

In [214]:
filtered_train.shape, filtered_val.shape

((133500, 25), (126081, 25))

In [215]:
#min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = 0
#filtered_train.loc[:,'semana_new'] = filtered_train.mes * 5 + filtered_train.semana

#min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = 0
#filtered_val.loc[:,'semana_new'] = filtered_val.mes * 5 + filtered_val.semana

In [216]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_634282/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_634282/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [217]:
label_week1 = '0_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '0_2'
label_week3 = '0_3'
label_week4 = '0_4'
label_week5 = '0_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  30176
Número de labels disponíveis para a semana 2:  41329
Número de labels disponíveis para a semana 3:  29503
Número de labels disponíveis para a semana 4:  30877
Número de labels disponíveis para a semana 5:  1615


In [218]:
features_train = aggr_df(filtered_train, month=0)
features_val = aggr_df(filtered_val, month=0)

In [219]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [220]:
features_train.target_week1.value_counts();

In [320]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000798 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5713
[LightGBM] [Info] Number of data points in the train set: 7311, number of used features: 55
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 5.536932
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[155]	valid_0's l2: 23.191	valid_0's wmape: 0.403087
New best wMAPE: 0.4031 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise

In [321]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 20,
 'min_child_samples': 5,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [221]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 20,
 'min_child_samples': 5,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [ ]:
params = 

In [222]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001903 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6236
[LightGBM] [Info] Number of data points in the train set: 30176, number of used features: 72
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.613319
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[50]	valid_0's l2: 94.8114	valid_0's wmape: 0.444373
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set 

In [223]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.49737208945553), np.float64(0.055482116415542716))

In [224]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4969626842574453), np.float64(0.05872406170725759))

# EXP3 - 3 meses

In [52]:
filtered_train = filtered[filtered.mes.isin([7,8,9])]
filtered_val = filtered[filtered.mes.isin([9,10,11])]

In [53]:
min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = filtered_train.mes - min_month_train

min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = filtered_val.mes - min_month_val

In [54]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_586632/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_586632/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [55]:
label_week1 = '2_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '2_2'
label_week3 = '2_3'
label_week4 = '2_4'
label_week5 = '2_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  39837
Número de labels disponíveis para a semana 2:  53189
Número de labels disponíveis para a semana 3:  40102
Número de labels disponíveis para a semana 4:  41297
Número de labels disponíveis para a semana 5:  1861


In [56]:
features_train = aggr_df(filtered_train, month=2)
features_val = aggr_df(filtered_val, month=2)

In [57]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [202]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19546
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 134
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[62]	valid_0's l2: 69.0324	valid_0's wmape: 0.434832
New best wMAPE: 0.4348 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-w

In [203]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.01,
 'reg_lambda': 0.01}

In [58]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.01,
 'reg_lambda': 0.01}

In [60]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005690 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19526
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 124
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[40]	valid_0's l2: 70.3131	valid_0's wmape: 0.42318
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005433 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20059
[Ligh

In [61]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.45659816707165685), np.float64(0.055305939173444285))

In [62]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4481923345030996), np.float64(0.0566297438084312))

# EXP4 - 4 meses

In [63]:
filtered_train = filtered[filtered.mes.isin([6,7,8,9])]
filtered_val = filtered[filtered.mes.isin([8,9,10,11])]

In [64]:
min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = filtered_train.mes - min_month_train

min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = filtered_val.mes - min_month_val

In [65]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_586632/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_586632/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [66]:
label_week1 = '3_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '3_2'
label_week3 = '3_3'
label_week4 = '3_4'
label_week5 = '3_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  39837
Número de labels disponíveis para a semana 2:  53189
Número de labels disponíveis para a semana 3:  40102
Número de labels disponíveis para a semana 4:  41297
Número de labels disponíveis para a semana 5:  1861


In [67]:
features_train = aggr_df(filtered_train, month=3)
features_val = aggr_df(filtered_val, month=3)

In [68]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [213]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26201
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 164
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[55]	valid_0's l2: 68.4114	valid_0's wmape: 0.441474
New best wMAPE: 0.4415 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-w

In [216]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.1,
 'reg_lambda': 0.01}

In [69]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.1,
 'reg_lambda': 0.01}

In [70]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26181
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 154
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[51]	valid_0's l2: 71.0005	valid_0's wmape: 0.433397
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can se

In [71]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.47502699900639306), np.float64(0.04855968305791016))

In [72]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4669428822135238), np.float64(0.05086563001037789))

**Resultados EXP1 (2 meses):**
WMAPE = 46.23% +- 3.70%

**Resultados EXP2 (1 mes):**
WMAPE = 47.73 +- 4.05%

**Resultados EXP3 (3 meses):**
WMAPE = 45.66 +- 5.53%

**Resultados EXP4 (4 meses):**
WMAPE = 47.5% +- 4.86%

# Com o melhor resultado (EXP1), vamos fazer um treinamento com os últimos dois meses disponíveis:

In [38]:
filtered_train = filtered[filtered.mes.isin([10,11])]
test = filtered[filtered.mes.isin([11,12])]

In [39]:
#values = (test.pdv_produto.value_counts() >= 2).values
#test = test[test['pdv_produto'].isin(test.pdv_produto.value_counts()[values].index)]

In [40]:
test.pdv_produto.nunique()

152847

In [41]:
min_month_train = filtered_train.mes.min()
filtered_train['mes'] = filtered_train.mes - min_month_train

min_month_test = test.mes.min()
test['mes'] = test.mes - min_month_test

/tmp/ipykernel_653421/3253973998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['mes'] = filtered_train.mes - min_month_train
/tmp/ipykernel_653421/3253973998.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mes'] = test.mes - min_month_test


In [42]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
test['semana_mes'] =  test['mes'].astype(str) + "_" + test['semana'].astype(str)

/tmp/ipykernel_653421/3447113126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/3447113126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['semana_mes'] =  test['mes'].astype(str) + "_" + test['semana'].astype(str)


In [43]:
label_week1 = '1_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '1_2'
label_week3 = '1_3'
label_week4 = '1_4'
label_week5 = '1_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  32245
Número de labels disponíveis para a semana 2:  34224
Número de labels disponíveis para a semana 3:  37212
Número de labels disponíveis para a semana 4:  19458
Número de labels disponíveis para a semana 5:  2942


In [44]:
features_train = aggr_df(filtered_train)
features_test = aggr_df(test)

In [45]:
features_test.shape

(152847, 129)

In [46]:
features_train, features_test = combine_cat_cols(features_train, features_test)

In [47]:
models_2nd_deploy = []
wmapes_train_2nd_deploy = []
#predictions_2nd_deploy = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, params= params)

    models_2nd_deploy.append(model)
    preds = model.predict(X_train_week)
    #predictions_2nd_deploy.append(preds)
    wmapes_train_2nd_deploy.append(np.sum(np.abs(y_train_week - np.round(model.predict(X_train_week)))) / np.sum(np.abs(y_train_week)))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003300 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12655
[LightGBM] [Info] Number of data points in the train set: 32245, number of used features: 104
[LightGBM] [Info] Start training from score 5.407755
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003360 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12996
[LightGBM] [Info] Number of data points in the train set: 34224, number of used features: 104
[LightGBM] [Info] Start training from score 5.725133
Training model for week 3
[LightGBM] [Warning] Found whitespace in feature_names, replace with under

In [48]:
models

[LGBMRegressor(max_depth=30, metric=None, min_child_samples=5, n_estimators=200,
               n_jobs=16, num_leaves=20, objective='regression', random_state=41,
               reg_alpha=0.1, reg_lambda=0.1),
 LGBMRegressor(max_depth=30, metric=None, min_child_samples=5, n_estimators=200,
               n_jobs=16, num_leaves=20, objective='regression', random_state=41,
               reg_alpha=0.1, reg_lambda=0.1),
 LGBMRegressor(max_depth=30, metric=None, min_child_samples=5, n_estimators=200,
               n_jobs=16, num_leaves=20, objective='regression', random_state=41,
               reg_alpha=0.1, reg_lambda=0.1),
 LGBMRegressor(max_depth=30, metric=None, min_child_samples=5, n_estimators=200,
               n_jobs=16, num_leaves=20, objective='regression', random_state=41,
               reg_alpha=0.1, reg_lambda=0.1),
 LGBMRegressor(max_depth=30, metric=None, min_child_samples=5, n_estimators=200,
               n_jobs=16, num_leaves=20, objective='regression', random_state=4

In [49]:
wmapes_train_2nd_deploy

[np.float64(0.3287353792219243),
 np.float64(0.3373749582161186),
 np.float64(0.3480923191673727),
 np.float64(0.30334937090559994),
 np.float64(0.22078827051008393)]

In [52]:
features_test.quantidade_0_1.value_counts()

quantidade_0_1
0.000000      120602
1.000000        8162
2.000000        7076
3.000000        4022
4.000000        2760
               ...  
9.750000           1
9.833333           1
4.979167           1
190.000000         1
92.000000          1
Name: count, Length: 163, dtype: int64

In [56]:
features_test.quantidade_1_4.value_counts()

quantidade_1_4
0.0      94554
1.0      13224
2.0      11454
3.0       6245
4.0       4066
         ...  
9.8          1
64.0         1
73.0         1
81.0         1
114.0        1
Name: count, Length: 206, dtype: int64

In [57]:
features_test.shape

(152847, 129)

In [59]:
X = features_test.drop(columns=target_cols + ['pdv_produto'])
outputs = []
for i in range(5):
    model = models_2nd_deploy[i]
    preds = np.round(model.predict(X))
    outputs.append(preds)

In [60]:
#data['pdv_produto'] = data['pdv'].astype(str) + '_' + data['produto'].astype(str)

In [61]:
pdv = features_test.pdv_produto.apply(lambda x: int(x.split('_')[0]))
produto = features_test.pdv_produto.apply(lambda x: int(x.split('_')[1]))

In [62]:
pdv.shape

(152847,)

In [63]:
pred_dfs = []
for i in range(5):
    pred_df = pd.DataFrame({
        'pdv': pdv,
        'produto': produto,
        'semana': i + 1,
        'quantidade': outputs[i]
    })
    pred_dfs.append(pred_df)

final_pred_df = pd.concat(pred_dfs, axis=0)

In [64]:
final_pred_df.reset_index(drop=True, inplace=True)
final_pred_df

,pdv,produto,semana,quantidade
0,1001371918471115422,1029370090212151375,1,3.0
1,1001371918471115422,1120490062981954254,1,18.0
2,1001371918471115422,1394381856358939027,1,5.0
3,1001371918471115422,1454838625590783593,1,4.0
4,1001371918471115422,2097372625156623809,1,2.0
...,...,...,...,...
764230,997907800111849739,8948552096954692956,5,9.0
764231,997907800111849739,9036639434778896273,5,4.0
764232,997907800111849739,9114472014485214982,5,32.0
764233,997907800111849739,9149212002603716850,5,5.0


In [65]:
final_pred_df.quantidade.value_counts()

quantidade
 4.0      126554
 2.0       86853
 8.0       65104
 5.0       59531
 3.0       56649
           ...  
 245.0         1
 360.0         1
-44.0          1
-31.0          1
 424.0         1
Name: count, Length: 330, dtype: int64

In [66]:
final_pred_df.quantidade = final_pred_df.quantidade.astype(int)

In [67]:
final_pred_df[['semana', 'pdv', 'produto', 'quantidade']]

,semana,pdv,produto,quantidade
0,1,1001371918471115422,1029370090212151375,3
1,1,1001371918471115422,1120490062981954254,18
2,1,1001371918471115422,1394381856358939027,5
3,1,1001371918471115422,1454838625590783593,4
4,1,1001371918471115422,2097372625156623809,2
...,...,...,...,...
764230,5,997907800111849739,8948552096954692956,9
764231,5,997907800111849739,9036639434778896273,4
764232,5,997907800111849739,9114472014485214982,32
764233,5,997907800111849739,9149212002603716850,5


In [68]:
final_pred_df.semana.value_counts(), final_pred_df.pdv.nunique(), final_pred_df.produto.nunique(), final_pred_df.shape

(semana
 1    152847
 2    152847
 3    152847
 4    152847
 5    152847
 Name: count, dtype: int64,
 8710,
 2461,
 (764235, 4))

In [ ]:
import autorootcwd #noqa 

deploy1 = pd.read_parquet("submissions/submission_model_v1.parquet")
final_pred_df.merge(deploy1, on=['pdv', 'produto', 'semana'], how='inner')

,pdv,produto,semana,quantidade_x,quantidade_y
0,1001371918471115422,1029370090212151375,1,3,4
1,1001371918471115422,1120490062981954254,1,18,17
2,1001371918471115422,1454838625590783593,1,4,4
3,1001371918471115422,2097372625156623809,1,2,2
4,1001371918471115422,2239307647969388381,1,9,9
...,...,...,...,...,...
616560,997907800111849739,7029448080469668619,5,3,4
616561,997907800111849739,7162840612005571065,5,9,12
616562,997907800111849739,802532509492578586,5,5,16
616563,997907800111849739,9036639434778896273,5,4,7


In [ ]:

final_pred_df.to_parquet("submissions/submission_model_v1.parquet", index=False)

In [246]:
final_pred_df.to_csv("submissions/submission_model_v1.csv", index=False, encoding='utf-8', sep=';')